## Particionando Dataframes
Neste notebook iremos aprender a particionar dados armazenados em Dataframes!

### Importando as bibliotecas
Nesta etapa iremos apenas importar todas as bibliotecas e funções necessárias para rodar o programa

In [3]:
from pyspark.sql import SparkSession

### Criando uma SparkSession
Por meio de uma SparkSession terei acesso ao SparkContext da minha aplicação.

In [5]:
spark = SparkSession.builder.appName('Select').getOrCreate()

### Carregando os dados
Nesta etapa estamos carregando os dados que utilizaremos neste notebook

In [7]:
dados = spark \
    .read.option("header","true") \
    .option("inferSchema","true") \
    .option("delimiter",";") \
    .format("csv") \
    .load("/FileStore/tables/bank_additional_full-3fd09.csv")

### Consulta o número de partições

In [9]:
dados.rdd.getNumPartitions()

Out[19]: 2

**Consulta o número de registros em cada partição**

In [11]:
dados.rdd.glom().map(lambda x: len(x)).collect()

Out[33]: [29543, 11645]

### Reduzindo o número de partições de um dataframe

In [13]:
dados1 = dados.coalesce(1)

In [14]:
dados1.rdd.getNumPartitions()

Out[35]: 1

In [15]:
dados1.rdd.glom().map(lambda x: len(x)).collect()

Out[36]: [41188]

### Aumentando o número de partições

In [17]:
dados2 = dados.repartition(4)

In [18]:
dados2.rdd.getNumPartitions()

Out[38]: 4

In [19]:
dados2.rdd.glom().map(lambda x: len(x)).collect()

Out[39]: [10297, 10297, 10296, 10298]

### Diferença entre coalesce e repartition
Coalesce vai tentar organizar os dados das partições existentes apenas transferindo dados de algumas partições (que serão eliminadas)
para outras (que serão incrementadas). Não é necessário uma operação de full shuffle para reorganizar os dados e por isso é mais rápido do que o repartition.
<p>Por este mesmo motivo, não é possível usar o coalesce para aumentar o número de partições.
Já o repartition faz um full suffle nas partições para reorganizá-las.

### Criando partições com base no valor de uma coluna.
Por padrão, quando fazemos particionamento por coluna, Spark vai criar 200 partições.<br>
Neste caso, temos uma partição para cada valor da coluna "marital" e o restante são partições vazias

In [22]:
dados3 = dados.repartition("marital")

In [23]:
dados3.rdd.getNumPartitions()

Out[41]: 200

In [24]:
dados3.rdd.glom().map(lambda x: len(x)).collect()

Out[42]: [0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 80,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 4612,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 24928,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 11568,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

**Se você quiser, pode dar um coalesce para "countDistinct(marital) + 1" para diminuir o número de particoes e manter seu particionamento por "marital"**

In [26]:
dados4 = dados3.coalesce(5)

In [27]:
dados4.rdd.glom().map(lambda x: len(x)).collect()

Out[44]: [80, 4612, 24928, 11568, 0]

### Considerações sobre particionamento

Em uma situação normal de um ambiente de cluster, Spark não vai tentar usar todos os recursos disponíveis para
executar suas operações. Isso só vai acontecer se você configurar o nível de paralelismo de suas operações com um valor
alto suficiente para que o cluster seja utilizado por inteiro.

<p>Por exemplo, Spark define automaticamente o número de tarefas map para rodar em um determinado arquivo de acordo com o
seu tamanho (embora seja possível configurar este comportamento por meio do parâmetro opcional SparkContext.textFile).
Já para operações de reduce Spark automaticamente usa a quantidade de partições do maior RDD envolvido na operação.

<p>Você pode configurar o nível de paralelismo do Spark por meio da configuração spark.PairRDDFunctions ou por meio da
propriedade spark.default.parallelism

<p>Na documentação do Spark é recomendado 1-3 tarefas por CPU core existente no cluster.
Você pode usar esta lógica para particionar seus dados e tomar melhor proveito do cluster.

<p>ou seja,

<p>n_partitions = n_cpu_cores * 3

<p>Por fim, mais uma dica sobre particionamento:
Logo após filtrar um dataframe com grande volume de dados, você deve pensar se deve reparticionar o seu dataframe menor ou não
(em muitos casos vale a pena reorganizar as partições para obter ganhos de desempenho)

Parte da lógica desta aula teve como base o seguinte artigo:
https://medium.com/@mrpowers/managing-spark-partitions-with-coalesce-and-repartition-4050c57ad5c4

### Obrigado!
Quer construir uma carreira em Data Science? Acesse meu blog pessoal em https://www.hackinganalytics.com/